In [21]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sqlite3

pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

def vc(ds, c):
    print(ds[c].value_counts(dropna=False))
    
def run_query(q):
    with sqlite3.connect('mlb.db') as conn:
        return pd.read_sql(q, conn)
    
def add_table(d, n):
    with sqlite3.connect('mlb.db') as conn:
        d.to_sql(n, conn, index=False)
    
def run_command(q):
    with sqlite3.connect('mlb.db') as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(q)
        
def show_tables():
    return run_query("""SELECT name, type
                        FROM sqlite_master
                        WHERE type IN ('table', 'view')
                        """)

def drop_table(n):
    run_command('DROP TABLE {}'.format(n))

In [22]:
games = pd.read_csv('game_log.csv')
parks = pd.read_csv('park_codes.csv')
persons = pd.read_csv('person_codes.csv')
teams = pd.read_csv('team_codes.csv')

/dataquest/system/env/python3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (12,13,14,15,19,20,81,82,83,84,85,86,87,88,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


The following warning arises after running the code for the games dataframe:

DtypeWarning: Columns (12,13,14,15,19,20,81,82,83,84,85,86,87,88,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)

After the review it was revealed that all values in these columns should be converted to object data type. 

In [23]:
mixed_dtypes_cols = list(games.dtypes.iloc[[
    12,13,14,15,19,20,81,
    82,83,84,85,86,87,88,
    93,94,95,96,97,98,99,
    100,105,106,108,109,
    111,112,114,115,117,
    118,120,121,123,124,
    126,127,129,130,132,
    133,135,136,138,139,
    141,142,144,145,147,
    148,150,151,153,154,
    156,157,160]].index)

games[mixed_dtypes_cols] = games[
    mixed_dtypes_cols].astype('object')

We will explore information about dataframes.

In [24]:
games.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,NaN,1,FW1,NaN,1,0,2,54.0,D,NaN,NaN,NaN,FOR01,200.0,120.0,000000000,010010000,30.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,6.0,1.0,NaN,-1.0,NaN,4.0,1.0,1.0,1.0,0.0,0.0,27.0,9.0,0.0,3.0,0.0,0.0,31.0,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,NaN,-1.0,NaN,3.0,1.0,0.0,0.0,0.0,0.0,27.0,3.0,3.0,1.0,1.0,0.0,boakj901,John Boake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,NaN,NaN,NaN,NaN,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2.0,kimbg101,Gene Kimball,4.0,paboc101,Charlie Pabor,7.0,allia101,Art Allison,8.0,white104,Elmer White,9.0,prata101,Al Pratt,1.0,sutte101,Ezra Sutton,5.0,carlj102,Jim Carleton,3.0,bassj101,John Bass,6.0,selmf101,Frank Sellman,5.0,mathb101,Bobby Mathews,1.0,foraj101,Jim Foran,3.0,goldw101,Wally Goldsmith,6.0,lennb101,Bill Lennon,2.0,caret101,Tom Carey,4.0,mince101,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,NaN,1,WS3,NaN,1,20,18,54.0,D,NaN,NaN,NaN,WAS01,5000.0,145.0,107000435,640113030,41.0,13.0,1.0,2.0,0.0,13.0,0.0,0.0,0.0,18.0,NaN,5.0,3.0,NaN,-1.0,NaN,12.0,1.0,6.0,6.0,1.0,0.0,27.0,13.0,10.0,1.0,2.0,0.0,49.0,14.0,2.0,0.0,0.0,11.0,0.0,0.0,0.0,10.0,NaN,2.0,1.0,NaN,-1.0,NaN,14.0,1.0,7.0,7.0,0.0,0.0,27.0,20.0,10.0,2.0,3.0,0.0,dobsh901,Henry Dobson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,NaN,NaN,NaN,NaN,spala101,Al Spalding,braia102,Asa Brainard,wrigg101,George Wright,6.0,barnr102,Ross

In [25]:
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


In [26]:
parks.head()

,park_id,name,aka,city,state,start,end,league,notes
0,ALB01,Riverside Park,NaN,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,NaN,Altoona,PA,04/30/1884,05/31/1884,UA,NaN
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,04/19/1966,NaN,AL,NaN
3,ARL01,Arlington Stadium,NaN,Arlington,TX,04/21/1972,10/03/1993,AL,NaN
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,04/11/1994,NaN,AL,NaN


There is a column 'park_id' in 'games' dataset which intersects with column 'park_id' in 'parks' dataset. 

In [27]:
persons.head()

,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
0,aardd001,Aardsma,David,04/06/2004,NaN,NaN,NaN
1,aaroh101,Aaron,Hank,04/13/1954,NaN,NaN,NaN
2,aarot101,Aaron,Tommie,04/10/1962,NaN,04/06/1979,NaN
3,aased001,Aase,Don,07/26/1977,NaN,NaN,NaN
4,abada001,Abad,Andy,09/10/2001,NaN,NaN,NaN


There is a column 'id' in 'persons' dataset which might intersect with the following columns in 'games' dataset:
- 'hp_umpire_id';
- '1b_umpire_id';
- '2b_umpire_id';
- '3b_umpire_id';
- 'lf_umpire_id';
- 'rf_umpire_id';
- 'v_manager_id';
- 'h_manager_id';
- 'winning_pitcher_id';
- 'losing_pitcher_id';
- 'saving_pitcher_id';
- 'winning_rbi_batter_id';
- 'v_starting_pitcher_id';
- 'h_starting_pitcher_id';
- 'v_player_k_id' (k stands for players number 1-9);
- 'h_player_k_id' (k stands for players number 1-9);

In [28]:
teams.head()

,team_id,league,start,end,city,nickname,franch_id,seq
0,ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1
1,ARI,NL,1998,0,Arizona,Diamondbacks,ARI,1
2,BFN,NL,1879,1885,Buffalo,Bisons,BFN,1
3,BFP,PL,1890,1890,Buffalo,Bisons,BFP,1
4,BL1,NaN,1872,1874,Baltimore,Canaries,BL1,1


There is a column 'team_id' in 'teams' dataset which intersects with the following columns in 'games' dataset:
- 'v_name';
- 'h_name';

There is some additional information about defensive positions and leagues.

Defenition of each defensive position number:
1 - pitcher;
2 - catcher;
3 - first baseman;
4 - second baseman;
5 - third baseman;
6 - shortstop;
7 - left fielder;
8 - center fielder;
9 - right fielder.

Definition of values in league fields:
NL - National league (included in Major League Baseball);
AL - American league (included in Major League Baseball);
AA - American association (was included in Major League Baseball, however, the league went defunct prior to 1892);
FL - Federal league (was included in Major League Baseball, however, the league and all of its franchises went defunct after the 1915 season);
PL - Player's league (was included in Major League Baseball, however, the league went defunct prior to 1892);
UA - Union association (was included in Major League Baseball, however, the league went defunct prior to 1892);

In [29]:
data = [(games, 'game_log'), (parks, 'park_codes'),
       (persons, 'person_codes'), (teams, 'team_codes')]

for d, n in data:
    add_table(d, n)

show_tables()

,name,type
0,game_log,table
1,park_codes,table
2,person_codes,table
3,team_codes,table
4,person,table
5,park,table
6,app_type,table
7,appearance_type,table
8,league,table
9,game,table


In [30]:
q1 = """
ALTER TABLE game_log
ADD COLUMN game_id
"""
run_command(q1)

q2 = """
UPDATE game_log
SET game_id = h_name || date || number_of_game
"""
run_command(q2)

Opportunities to normalize the data:
- columns:
    - hp_umpire_name,
    - 1b_umpire_name,
    - 2b_umpire_name,
    - 3b_umpire_name,
    - lf_umpire_name,
    - rf_umpire_name,
    - v_manager_name,
    - h_manager_name,
    - winning_pitcher_name,
    - losing_pitcher_name,
    - saving_pitcher_name,
    - winning_rbi_batter_name,
    - v_starting_pitcher_name,
    - h_starting_pitcher_name,
    - v_player_k_name (k stands for players number 1-9),
    - h_player_k_name (k stands for players number 1-9) can be removed as this information contains in persons_codes table;
- create new table person_game which links different roles with each game. h_or_v column indicate whether the player is in home of visitor team. Columns game_id and id will be used as compound primary key. Remove corresponding columns from game_log table;
- create new table team_game which contains information about each team in every game. Columns game_id and name will be used as compound primary key. Remove corresponding columns from game_log table;
The proposed schema was attached to the repository. 

The rest of the project will be done according to the schema proposed by Data Quest which is also attached to the repository. We will now create the tables mentioned in the schema. 

In [31]:
# Create table person
q3 = """
CREATE TABLE IF NOT EXISTS person
    (
    person_id TEXT PRIMARY KEY,
    first_name TEXT,
    last_name TEXT
    )
"""
run_command(q3)

q4 = """
INSERT OR IGNORE INTO person
SELECT
    id,
    first,
    last
FROM person_codes
"""
run_command(q4)

q5 = """
SELECT * FROM person
LIMIT 10
"""
run_query(q5)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad
5,abadf001,Fernando,Abad
6,abadj101,John,Abadie
7,abbae101,Ed,Abbaticchio
8,abbeb101,Bert,Abbey
9,abbec101,Charlie,Abbey


In [32]:
# Create table park
q6 = """
CREATE TABLE IF NOT EXISTS park
    (
    park_id TEXT PRIMARY KEY,
    name TEXT,
    nickname TEXT,
    city TEXT,
    state TEXT,
    notes TEXT
    )
"""
run_command(q6)

q7 = """  
INSERT OR IGNORE INTO park
SELECT
    park_id,
    name,
    aka,
    city,
    state,
    notes
FROM park_codes;
"""
run_command(q7)

q8 = """
SELECT * FROM park
"""
run_query(q8)

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None
5,ATL01,Atlanta-Fulton County Stadium,None,Atlanta,GA,None
6,ATL02,Turner Field,None,Atlanta,GA,None
7,ATL03,Suntrust Park,None,Atlanta,GA,None
8,BAL01,Madison Avenue Grounds,None,Baltimore,MD,WS3
9,BAL02,Newington Park,None,Baltimore,MD,BL1:1872-74; BL4:1873; BL2: 1882


In [33]:
# Create table appearance_type
app_type = pd.read_csv('appearance_type.csv')
add_table(app_type, 'app_type')

q9 = """
CREATE TABLE IF NOT EXISTS appearance_type
    (
    appearance_type_id TEXT PRIMARY KEY,
    name TEXT,
    category TEXT)
"""
run_command(q9)

q10 = """
INSERT OR IGNORE INTO appearance_type
SELECT
    appearance_type_id,
    name,
    category
FROM app_type
"""

run_command(q10)

q11 = "SELECT * FROM appearance_type"
run_query(q11)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense
5,O6,Batter 6,offense
6,O7,Batter 7,offense
7,O8,Batter 8,offense
8,O9,Batter 9,offense
9,D1,Pitcher,defense


In [34]:
# Create table league
q12 = """
CREATE TABLE IF NOT EXISTS league
    (
    league_id TEXT PRIMARY KEY,
    name TEXT
    )
"""
run_command(q12)

q13 = """
INSERT OR IGNORE INTO league
VALUES
('NL', 'National league'),
('AL', 'American league'),
('AA', 'American association'),
('FL', 'Federal league'),
('PL', "Player's league"),
('UA', 'Union Association')
"""
run_command(q13)

q14 = "SELECT * FROM league"
run_query(q14)

,league_id,name
0,NL,National league
1,AL,American league
2,AA,American association
3,FL,Federal league
4,PL,Player's league
5,UA,Union Association


In [35]:
# Create table game
q15 = """
CREATE TABLE IF NOT EXISTS game
    (
    game_id TEXT PRIMARY KEY,
    date INTEGER,
    number_of_game INTEGER,
    park_id TEXT,
    length_outs REAL,
    day TEXT,
    completion TEXT,
    forfeit TEXT,
    protest TEXT,
    attendance REAL,
    length_minutes REAL,
    additional_info TEXT,
    acquisition_info TEXT,
    FOREIGN KEY (park_id) REFERENCES park(park_id)
    )
"""
run_command(q15)

q16 = """
INSERT OR IGNORE INTO game
SELECT
    game_id,
    date,
    number_of_game,
    park_id,
    length_outs,
    day_night,
    completion,
    forefeit,
    protest,
    attendance,
    length_minutes,
    additional_info,
    acquisition_info
FROM game_log
"""
run_command(q16)

q17 = "SELECT * FROM game LIMIT 10"
run_query(q17)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forfeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,FOR01,54.0,D,None,None,None,200.0,120.0,None,Y
1,WS3187105050,18710505,0,WAS01,54.0,D,None,None,None,5000.0,145.0,HTBF,Y
2,RC1187105060,18710506,0,RCK01,54.0,D,None,None,None,1000.0,140.0,None,Y
3,CH1187105080,18710508,0,CHI01,54.0,D,None,None,None,5000.0,150.0,None,Y
4,TRO187105090,18710509,0,TRO01,54.0,D,None,None,None,3250.0,145.0,HTBF,Y
5,CL1187105110,18710511,0,CLE01,48.0,D,None,V,None,2500.0,120.0,None,Y
6,CL1187105130,18710513,0,CIN01,54.0,D,None,None,None,1200.0,150.0,None,Y
7,FW1187105130,18710513,0,FOR01,54.0,D,None,None,None,1500.0,105.0,None,Y
8,FW1187105150,18710515,0,FOR01,54.0,D,None,None,None,NaN,140.0,None,Y
9,BS1187105160,18710516,0,BOS01,54.0,D,None,None,None,2500.0,NaN,HTBF,Y


In [36]:
# Create table team
q18 = """
CREATE TABLE IF NOT EXISTS team
    (
    team_id TEXT PRIMARY KEY,
    league_id TEXT,
    city TEXT,
    nickname TEXT,
    franch_id,
    FOREIGN KEY (league_id) REFERENCES league(league_id)
    )
"""
run_command(q18)

q19 = """
INSERT OR IGNORE INTO team
SELECT
    team_id,
    league,
    city,
    nickname,
    franch_id
FROM team_codes   
"""
run_command(q19)

q20 = "SELECT * FROM team"
run_query(q20)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1
5,BL2,AA,Baltimore,Orioles,BL2
6,BLN,NL,Baltimore,Orioles,BL2
7,BL4,None,Baltimore,Marylands,BL4
8,BLA,AL,Baltimore,Orioles,BLA
9,NYA,AL,New York,Yankees,BLA


In [38]:
# Display sqlite master table to copy and past name and
# data types of columns for team_appearance table
q21 = """
SELECT sql 
FROM sqlite_master
WHERE name = "game_log"
AND type = "table"
"""
run_query(q21)

sql
0  CREATE TABLE "game_log" (\n"date" INTEGER,\n  "number_of_game" INTEGER,\n  "day_of_week" TEXT,\n  "v_name" TEXT,\n  "v_league" TEXT,\n  "v_game_number" INTEGER,\n  "h_name" TEXT,\n  "h_league" TEXT,\n  "h_game_number" INTEGER,\n  "v_score" INTEGER,\n  "h_score" INTEGER,\n  "length_outs" REAL,\n  "day_night" TEXT,\n  "completion" TEXT,\n  "forefeit" TEXT,\n  "protest" TEXT,\n  "park_id" TEXT,\n  "attendance" REAL,\n  "length_minutes" REAL,\n  "v_line_score" TEXT,\n  "h_line_score" TEXT,\n  "v_at_bats" REAL,\n  "v_hits" REAL,\n  "v_doubles" REAL,\n  "v_triples" REAL,\n  "v_homeruns" REAL,\n  "v_rbi" REAL,\n  "v_sacrifice_hits" REAL,\n  "v_sacrifice_flies" REAL,\n  "v_hit_by_pitch" REAL,\n  "v_walks" REAL,\n  "v_intentional_walks" REAL,\n  "v_strikeouts" REAL,\n  "v_stolen_bases" REAL,\n  "v_caught_stealing" REAL,\n  "v_grounded_into_double" REAL,\n  "v_first_catcher_interference" REAL,\n  "v_left_on_base" REAL,\n  "v_pitchers_used" REAL,\n  "v_individual_earned_runs" REAL,\n  "v_team_earned_runs" REAL,\n  "v_wild_pitches" REAL,\n  "v_balks" REAL,\n  "v_putouts" REAL,\n  "v_assists" REAL,\n  "v_errors" REAL,\n  "v_passed_balls" REAL,\n  "v_double_plays" REAL,\n  "v_triple_plays" REAL,\n  "h_at_bats" REAL,\n  "h_hits" REAL,\n  "h_doubles" REAL,\n  "h_triples" REAL,\n  "h_homeruns" REAL,\n  "h_rbi" REAL,\n  "h_sacrifice_hits" REAL,\n  "h_sacrifice_flies" REAL,\n  "h_hit_by_pitch" REAL,\n  "h_walks" REAL,\n  "h_intentional_walks" REAL,\n  "h_strikeouts" REAL,\n  "h_stolen_bases" REAL,\n  "h_caught_stealing" REAL,\n  "h_grounded_into_double" REAL,\n  "h_first_catcher_interference" REAL,\n  "h_left_on_base" REAL,\n  "h_pitchers_used" REAL,\n  "h_individual_earned_runs" REAL,\n  "h_team_earned_runs" REAL,\n  "h_wild_pitches" REAL,\n  "h_balks" REAL,\n  "h_putouts" REAL,\n  "h_assists" REAL,\n  "h_errors" REAL,\n  "h_passed_balls" REAL,\n  "h_double_plays" REAL,\n  "h_triple_plays" REAL,\n  "hp_umpire_id" TEXT,\n  "hp_umpire_name" TEXT,\n  "1b_umpire_id" TEXT,\n  "1b_umpire_name" TEXT,\n  "2b_umpire_id" TEXT,\n  "2b_umpire_name" TEXT,\n  "3b_umpire_id" TEXT,\n  "3b_umpire_name" TEXT,\n  "lf_umpire_id" TEXT,\n  "lf_umpire_name" TEXT,\n  "rf_umpire_id" TEXT,\n  "rf_umpire_name" TEXT,\n  "v_manager_id" TEXT,\n  "v_manager_name" TEXT,\n  "h_manager_id" TEXT,\n  "h_manager_name" TEXT,\n  "winning_pitcher_id" TEXT,\n  "winning_pitcher_name" TEXT,\n  "losing_pitcher_id" TEXT,\n  "losing_pitcher_name" TEXT,\n  "saving_pitcher_id" TEXT,\n  "saving_pitcher_name" TEXT,\n  "winning_rbi_batter_id" TEXT,\n  "winning_rbi_batter_id_name" TEXT,\n  "v_starting_pitcher_id" TEXT,\n  "v_starting_pitcher_name" TEXT,\n  "h_starting_pitcher_id" TEXT,\n  "h_starting_pitcher_name" TEXT,\n  "v_player_1_id" TEXT,\n  "v_player_1_name" TEXT,\n  "v_player_1_def_pos" REAL,\n  "v_player_2_id" TEXT,\n  "v_player_2_name" TEXT,\n  "v_player_2_def_pos" REAL,\n  "v_player_3_id" TEXT,\n  "v_player_3_name" TEXT,\n  "v_player_3_def_pos" REAL,\n  "v_player_4_id" TEXT,\n  "v_player_4_name" TEXT,\n  "v_player_4_def_pos" REAL,\n  "v_player_5_id" TEXT,\n  "v_player_5_name" TEXT,\n  "v_player_5_def_pos" REAL,\n  "v_player_6_id" TEXT,\n  "v_player_6_name" TEXT,\n  "v_player_6_def_pos" REAL,\n  "v_player_7_id" TEXT,\n  "v_player_7_name" TEXT,\n  "v_player_7_def_pos" REAL,\n  "v_player_8_id" TEXT,\n  "v_player_8_name" TEXT,\n  "v_player_8_def_pos" REAL,\n  "v_player_9_id" TEXT,\n  "v_player_9_name" TEXT,\n  "v_player_9_def_pos" REAL,\n  "h_player_1_id" TEXT,\n  "h_player_1_name" TEXT,\n  "h_player_1_def_pos" REAL,\n  "h_player_2_id" TEXT,\n  "h_player_2_name" TEXT,\n  "h_player_2_def_pos" REAL,\n  "h_player_3_id" TEXT,\n  "h_player_3_name" TEXT,\n  "h_player_3_def_pos" REAL,\n  "h_player_4_id" TEXT,\n  "h_player_4_name" TEXT,\n  "h_player_4_def_pos" REAL,\n  "h_player_5_id" TEXT,\n  "h_player_5_name" TEXT,\n  "h_player_5_def_pos" REAL,\n  "h_player_6_id" TEXT,\n  "h_player_6_name" TEXT,\n  "h_player_6_def_pos" REAL,\n  "h_player_7_id" TEXT,\n  "h_player_7_name" TEXT,\n  "h_playe

In [39]:
# Create table team_appearance

q22 = """
CREATE TABLE IF NOT EXISTS team_appearance
    (
    team_id TEXT,
    game_id TEXT,
    home INTEGER,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats REAL,
    hits REAL,
    doubles REAL,
    triples REAL,
    homeruns REAL,
    rbi REAL,
    sacrifice_hits REAL,
    sacrifice_flies REAL,
    hit_by_pitch REAL,
    walks REAL,
    intentional_walks REAL,
    strikeouts REAL,
    stolen_bases REAL,
    caught_stealing REAL,
    grounded_into_double REAL,
    first_catcher_interference REAL,
    left_on_base REAL,
    pitchers_used REAL,
    individual_earned_runs REAL,
    team_earned_runs REAL,
    wild_pitches REAL,
    balks REAL,
    putouts REAL,
    assists REAL,
    errors REAL,
    passed_balls REAL,
    double_plays REAL,
    triple_plays REAL,
    PRIMARY KEY (team_id, game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (league_id) REFERENCES league(league_id)
    )
"""
run_command(q22)

q23 = """
INSERT OR IGNORE INTO team_appearance
    SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays
    FROM game_log
    
UNION

    SELECT
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    FROM game_log;
"""
run_command(q23)

q24 = """
SELECT * FROM team_appearance LIMIT 100"""
run_query(q24)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,ALT,ALT188404300,1,UA,2,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALT,ALT188405020,1,UA,3,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ALT,ALT188405030,1,UA,5,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ALT,ALT188405050,1,UA,2,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ALT,ALT188405100,1,UA,9,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ALT,ALT188405120,1,UA,3,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ALT,ALT188405140,1,UA,2,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ALT,ALT188405150,1,UA,7,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ALT,ALT188405160,1,UA,6,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ALT,ALT188405170,1,UA,8,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
# Create table person_appearance

q25 = """
CREATE TABLE IF NOT EXISTS person_appearance
    (
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id TEXT,
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id)
    REFERENCES appearance_type(appearance_type_id)
    )
"""
run_command(q25)

# Column appearance_id will be autopopulated as the data
# type is integer

# Insert data on players
temp1 = """
INSERT OR IGNORE INTO person_appearance
    (
    person_id,
    team_id,
    game_id,
    appearance_type_id
    )
SELECT
    {hv}_player_{num}_id,
    {hv}_name,
    game_id,
    "O{num}"
FROM game_log
WHERE {hv}_player_{num}_id IS NOT NULL

UNION

SELECT
    {hv}_player_{num}_id,
    {hv}_name,
    game_id,
    "D" || CAST({hv}_player_{num}_def_pos AS INTEGER)
FROM game_log
WHERE {hv}_player_{num}_id IS NOT NULL
"""

for hv in ['h', 'v']:
    for num in range(1, 10):
        keys = {'hv': hv, 'num': num}
        run_command(temp1.format(**keys))

In [41]:
# Insert data on umpires
temp2 = """
INSERT OR IGNORE INTO person_appearance
    (
    person_id,
    team_id,
    game_id,
    appearance_type_id
    )
SELECT
    {code},
    NULL,
    game_id,
    "{ati}"
FROM game_log
WHERE {code} IS NOT NULL
"""
umpires = [
('hp_umpire_id', 'UHP'),
('[1b_umpire_id]', 'U1B'),
('[2b_umpire_id]', 'U2B'),
('[3b_umpire_id]', 'U3B'),
('lf_umpire_id', 'ULF'),
('rf_umpire_id', 'URF'),
]

for code, ati in umpires:
    keys = {'code': code, 'ati': ati}
    run_command(temp2.format(**keys))

In [47]:
# Insert data on pitchers, managers and winning batters
q26 = """
INSERT OR IGNORE INTO person_appearance
    (
    person_id,
    team_id,
    game_id,
    appearance_type_id
    )

	SELECT
		v_manager_id,
		v_name,
	    game_id,
	    "MM"
	FROM game_log
	WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        h_manager_id,
        h_name,
        game_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        winning_pitcher_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        game_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION
	
	SELECT
        losing_pitcher_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
            END,
        game_id,
        "AWP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL

UNION

	SELECT
		saving_pitcher_id,
		CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        game_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL

UNION

	SELECT
		winning_rbi_batter_id, 
		CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        game_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL

UNION

	SELECT
		v_starting_pitcher_id,
		v_name,
		game_id,
		"PSP"
	FROM game_log
	WHERE v_starting_pitcher_id IS NOT NULL

UNION

	SELECT
		h_starting_pitcher_id,
		h_name,
		game_id,
		"PSP"
	FROM game_log
	WHERE h_starting_pitcher_id IS NOT NULL	
"""
run_command(q26)

In [52]:
q27 = """
SELECT *
FROM person_appearance
GROUP BY appearance_type_id
"""
run_query(q27)

,appearance_id,person_id,team_id,game_id,appearance_type_id
0,17905404,zuveg101,DET,WS1195404300,ASP
1,17905408,zuvep001,CLE,SEA198909270,AWB
2,17905409,zycht001,SEA,CLE201604210,AWP
3,16259221,zuveg101,DET,WS1195408310,D1
4,16258433,zeilt001,NYA,OAK200308020,D10
5,16259189,zunim001,SEA,TEX201508190,D2
6,16258691,zinta001,HOU,TEX200209020,D3
7,16258945,zobrb001,TBA,OAK200805200,D4
8,16258425,zeilt001,NYA,CLE200307090,D5
9,16259291,zuvep001,NYA,TEX198607090,D6


In [53]:
show_tables()

,name,type
0,game_log,table
1,park_codes,table
2,person_codes,table
3,team_codes,table
4,person,table
5,park,table
6,app_type,table
7,appearance_type,table
8,league,table
9,game,table


In [54]:
# Remove tables which are not needed in the database
tables_to_drop = ['game_log', 'park_codes', 'team_codes',
                 'person_codes', 'app_type']

for t in tables_to_drop:
    drop_table(t)
show_tables()

,name,type
0,person,table
1,park,table
2,appearance_type,table
3,league,table
4,game,table
5,team,table
6,team_appearance,table
7,person_appearance,table
